In [1]:
%load_ext autoreload
%autoreload 2

# Plan

In [2]:
import syft as sy
from syft.core.node.common.action.get_object_action import GetObjectAction
from syft.core.node.common.action.function_or_constructor_action import RunFunctionOrConstructorAction
from syft.core.node.common.action.common import Action
from syft.proto.core.node.common.plan.plan_pb2 import Plan as Plan_PB

from typing import List
from syft.core.common.uid import UID
from syft.core.io.address import Address
from syft.core.common.serde.deserialize import _deserialize
from syft.core.common.object import Serializable
from syft.proto.core.node.common.action.action_pb2 import Action as Action_PB
import re
import torch as th
import sys

In [3]:
# %debug

In [4]:
from syft.core.node.common.plan.plan import Plan

In [5]:
address=Address()

In [6]:
args = (
    th.tensor([1, 2, 3]),
    th.tensor([4, 5, 5]),
)

In [7]:
a1 = GetObjectAction(id_at_location=UID(), address=address, reply_to=Address(), msg_id=UID())
a2 = RunFunctionOrConstructorAction(path="torch.Tensor.add",args=tuple(),kwargs={},id_at_location=UID(),
                                    address=address,msg_id=UID())

In [8]:
plan = Plan([a1,a2])

In [9]:
blob = plan.serialize()
msg2 = sy.deserialize(blob=blob)

In [10]:
assert isinstance(msg2, Plan)
assert all(isinstance(a, Action) for a in msg2.actions)

# Execution

In [11]:
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()

In [12]:
action = GetObjectAction(id_at_location=UID(), address=address, reply_to=Address(), msg_id=UID())

In [13]:
plan = Plan([action])

In [14]:
from syft.lib import create_lib_ast

In [15]:
plan_pointer = plan.send(alice_client)

In [16]:
plan_pointer

In [17]:
plan_pointer.execute()

AttributeError: 'StorableObject' object has no attribute 'actions'

In [19]:
%debug

> /Users/koen/workspace/PySyft/src/syft/core/node/common/plan/plan.py(23)execute()
     21 
     22     def execute(self):
---> 23         for a in self.actions:
     24             a.execute_action()
     25 

ipdb> self
<Storable: <syft.core.node.common.plan.plan.Plan object at 0x7f8a2df8e590>>
ipdb> self.actions
*** AttributeError: 'StorableObject' object has no attribute 'actions'
ipdb> self
<Storable: <syft.core.node.common.plan.plan.Plan object at 0x7f8a2df8e590>>
ipdb> dir(self)
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_data_object2proto', '_data_proto2object', '_description', '_object2proto', '_proto2object', '_tags', 'binary', 'class_name', 'cons

In [20]:
plan_pointer.execute

<bound method get_run_class_method.<locals>.run_class_method of <syft.proxy.syft.core.node.common.plan.plan.PlanPointer object at 0x7f930ac95c10>>

In [ ]:
action.core.node.common.plan.plan.Plan.execute